In [67]:
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns
from sklearn.model_selection import ShuffleSplit, GridSearchCV,train_test_split

from sksurv.datasets import load_veterans_lung_cancer
from sksurv.column import encode_categorical
from sksurv.metrics import concordance_index_censored
from sksurv.svm import FastSurvivalSVM,FastKernelSurvivalSVM
from sksurv.kernels import clinical_kernel

In [ ]:
all_features <- c(1:177)
eln_clin<-c(1,169:175)
eln_clin_demo<-c(1,169:177)
eln_clin_demo_cyto <-c(1,85:153,169:177)
eln_clin_demo_gen <-c(1:84,169:177)
eln_clin_demo_cyto_gen <-c(1:153,169:177)
eln_clin_demo_comp <-c(1,154:177)
eln_cyto_gen<-c(1:153)
eln_cyto_gen_comp <-c(1:168)
eln_cyto_comp <-c(1,85:168)
eln_gen_comp <-c(1:84,154:168)

clin_demo<-c(169:177)
clin_demo_cyto <-c(85:153,169:177)
clin_demo_gen <-c(2:84,169:177)
clin_demo_cyto_gen <-c(2:153,169:177)
clin_demo_comp <-c(154:177)
cyto_gen<-c(2:153)
cyto_gen_comp <-c(2:168)
cyto_comp <-c(85:168)
gen_comp <-c(2:84,154:168)
clin_demo_cyto_gen_comp<-c(2:177)
gen<-c(2:84)
cyto<-c(85:153)
comp<-c(154:168)

In [3]:
df_final = pd.read_table("df_prognosis_features_ready.tsv")
x = df_final.iloc[:,1:177]
#x = df_final.iloc[:,1:84]
y = np.array(list(zip(df_final.os_status, df_final.os)),dtype=[('Status', '?'), ('Survival_in_days', '<f8')])

In [49]:
{'alpha': 10. ** np.arange(-5.5, -4.5, 0.1)}
["a","b"]

['a', 'b']

In [50]:
#estimator = FastSurvivalSVM(optimizer="rbtree", max_iter=1000, tol=1e-6, random_state=17)
estimator = FastSurvivalSVM(max_iter=1000, tol=1e-6, random_state=17)

In [51]:
def score_survival_model(model, X, y):
    prediction = model.predict(X)
    result = concordance_index_censored(y['Status'], y['Survival_in_days'], prediction)
    return result[0]

In [82]:
param_grid = {'alpha': 10. ** np.arange(-6, -4, 0.1),'optimizer':["rbtree"]}
#param_grid = {'alpha': 10. ** np.arange(-6, -3, 0.5),'optimizer':
#param_grid = {'alpha': 10. ** np.arange(-10, -3, 0.2)}
#cv = ShuffleSplit(n_splits=5, test_size=0.5, random_state=0)
cv = ShuffleSplit(n_splits=5,random_state=17)
gcv = GridSearchCV(estimator, param_grid, scoring=score_survival_model,
                   n_jobs=4, iid=False, refit=True,
                   cv=cv)

In [ ]:
ci=[]
for i in range(25):
    X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=i)
    gcv = gcv.fit(X_train,y_train)
    ci.append(concordance_index_censored(y_test['Status'], y_test['Survival_in_days'], gcv.predict(X_test))[0])
#gcv = gcv.fit(clinical_kernel(x),y)


In [77]:
concordance_index_censored(y_test['Status'], y_test['Survival_in_days'], gcv.predict(X_test))

(0.7146985630569158, 44515, 17770, 0, 39)

In [55]:
gcv.best_score_

0.7205554260770012

In [56]:
gcv.best_params_

{'alpha': 1e-05, 'optimizer': 'rbtree'}

In [46]:
print(gcv.cv_results_)

{'mean_fit_time': array([1.9771034 , 3.02099858]), 'std_fit_time': array([0.11847194, 0.21321437]), 'mean_score_time': array([0.00434766, 0.00449839]), 'std_score_time': array([0.00037126, 0.00047645]), 'param_alpha': masked_array(data=[3.162277660168379e-06, 1e-05],
             mask=[False, False],
       fill_value='?',
            dtype=object), 'params': [{'alpha': 3.162277660168379e-06}, {'alpha': 1e-05}], 'split0_test_score': array([0.75801641, 0.75795426]), 'split1_test_score': array([0.73586812, 0.74275248]), 'split2_test_score': array([0.69129321, 0.69276569]), 'split3_test_score': array([0.72580131, 0.72134073]), 'split4_test_score': array([0.73887891, 0.74346699]), 'split5_test_score': array([0.70319326, 0.69919399]), 'split6_test_score': array([0.73891689, 0.73626729]), 'split7_test_score': array([0.73944451, 0.74206003]), 'split8_test_score': array([0.7257573, 0.7257573]), 'split9_test_score': array([0.72973698, 0.73268921]), 'split10_test_score': array([0.70290357, 0.703

In [38]:
#help(gcv)
print(gcv.cv_results_)

{'mean_fit_time': array([1.95929852, 2.15535703, 2.45642877, 2.82545433, 2.9712359 ,
       3.43145871, 3.6201467 , 3.76200402, 4.17489281, 4.35264575]), 'std_fit_time': array([0.09316165, 0.08327517, 0.18496588, 0.17330992, 0.12335856,
       0.09683836, 0.27536213, 0.14368053, 0.26379503, 0.39251015]), 'mean_score_time': array([0.00440171, 0.00466011, 0.0047617 , 0.00469139, 0.0047878 ,
       0.00503514, 0.00512285, 0.00497859, 0.00506957, 0.00484123]), 'std_score_time': array([0.00048422, 0.00047025, 0.00037283, 0.00040476, 0.00040411,
       0.00028405, 0.00022961, 0.00026873, 0.00037943, 0.00080902]), 'param_alpha': masked_array(data=[3.162277660168379e-06, 3.981071705534969e-06,
                   5.011872336272715e-06, 6.309573444801917e-06,
                   7.94328234724279e-06, 9.999999999999958e-06,
                   1.258925411794161e-05, 1.5848931924611043e-05,
                   1.9952623149688664e-05, 2.5118864315095615e-05],
             mask=[False, False, False, Fa

In [42]:
gcv.p

{'alpha': array([3.16227766e-06, 1.00000000e-05])}